In [1]:
import pandas as pd

In [13]:
# Load file with time component
df = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df.csv")

# Load file with time component, categories renamed
df_category = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_category.csv")

# Load file with time component, movie dummy
df_expanded_movies = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_expanded_movies.csv")

# Load file with time component, categories renamed, movie dummy
df_expanded_movies_category = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_expanded_movies_category.csv")

# Load FINAL file with time component, categories renamed, movie dummy, ordered right
df_final = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_final.csv")


In [26]:
df_final.head()

,UserID,MovieID,Rating,Year,Month,Date,Gender,Age,Occupation,Zip-code,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,2000,12,2000-12-31,F,Under 18,K-12 student,48067,...,False,False,False,False,False,False,False,False,False,False
1,2,1193,5,2000,12,2000-12-31,M,56+,self-employed,70072,...,False,False,False,False,False,False,False,False,False,False
2,12,1193,4,2000,12,2000-12-31,M,25-34,programmer,32793,...,False,False,False,False,False,False,False,False,False,False
3,15,1193,4,2000,12,2000-12-30,M,25-34,executive/managerial,22903,...,False,False,False,False,False,False,False,False,False,False
4,17,1193,5,2000,12,2000-12-30,M,50-55,academic/educator,95350,...,False,False,False,False,False,False,False,False,False,False


# Random forest

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
df_final.dtypes

UserID           int64
MovieID          int64
Rating           int64
Year             int64
Month            int64
Date            object
Gender          object
Age             object
Occupation      object
Zip-code        object
Title           object
Release_year     int64
Action            bool
Adventure         bool
Animation         bool
Children's        bool
Comedy            bool
Crime             bool
Documentary       bool
Drama             bool
Fantasy           bool
Film-Noir         bool
Horror            bool
Musical           bool
Mystery           bool
Romance           bool
Sci-Fi            bool
Thriller          bool
War               bool
Western           bool
dtype: object

In [18]:
# Define the target variable and features
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

In [21]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Initialize the Random Forest model
model = RandomForestClassifier(random_state=42)

In [23]:
# Train the model
model.fit(X_train, y_train)

ValueError: could not convert string to float: '2000-11-23'